In [10]:
import pickle
import pandas as pd
import numpy as np


from sklearn.utils import resample


df = pickle.load( open( "files/df.pickle", "rb" ) )
df['Tweet'] = df['Tweet'].astype(str)
df['Words'] = df['Tweet'].str.count(' ') + 1
df = df[df['Words']>=4]

print('tweets',df.shape[0])
print('Russian',np.sum(df['Category']))

tweets 8488073
Russian 2285628


In [11]:
df_majority = df[df['Category']==0]
df_minority = df[df['Category']==1]
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=df_minority.shape[0]    # to match minority class
                                 ) 

df_downsampled = pd.concat([df_majority_downsampled, df_minority])

In [12]:
cv = pickle.load( open( "models/countVectorizer.pickle", "rb" ) )

#cv0 = pickle.load( open( "models/countVectorizer1.pickle", "rb" ) )


countVector = cv.transform(df_downsampled['Processed'])
#countVector0 = cv1.transform(df_downsampled['Processed'])



In [4]:
lda = pickle.load( open( "models/lda_model.pickle", "rb" ) )
#lda1 = pickle.load( open( "models/lda_model1.pickle", "rb" ) )

ldaVector = lda.transform(countVector)
#ldaVector0 = lda0.transform(countVector)

#ldaVector = np.concatenate((ldaVector0,ldaVector1),axis=1)
#ldaVector = ldaVector1

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  1.6min finished


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(countVector, df_downsampled['Category'])

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
print(model.score(X_train, y_train), model.score(X_test, y_test))

In [7]:
pickle.dump( model, open( "files/model.pickle", "wb" ) )

In [8]:
model.coef_.shape

(1, 50)

In [9]:
X_test.shape

(1142814, 50)